# This script uses a fasta file and retrieve the most similar matches

Since BLAST fails to search for homologous sequences with query larger than ~200kb, first we will try to chunk long sequences larger than 100kb for optimal handling

In [11]:
#Import modules from biopython 1.76
from Bio import SeqIO
from Bio.Blast import NCBIWWW
from Bio import SearchIO
#from Bio.Blast.Applications import NcbiblastxCommandline

Let's explore how many records we will cut into pieces

In [139]:
fasta_to_chop = SeqIO.parse(open("TEST/KPN_FULL.fna"),"fasta")
for record_to_chop in fasta_to_chop:
    if len(record_to_chop) > 100000:
        print(record_to_chop.id, len(record_to_chop))
        #print(record_to_chop.seq[0:4])

gnl|BU-ISCIII|KPN05_000240185_1 502847
gnl|BU-ISCIII|KPN05_000240185_2 433789
gnl|BU-ISCIII|KPN05_000240185_3 327526
gnl|BU-ISCIII|KPN05_000240185_4 276424
gnl|BU-ISCIII|KPN05_000240185_5 271316
gnl|BU-ISCIII|KPN05_000240185_6 270429
gnl|BU-ISCIII|KPN05_000240185_7 260169
gnl|BU-ISCIII|KPN05_000240185_8 243764
gnl|BU-ISCIII|KPN05_000240185_9 235927
gnl|BU-ISCIII|KPN05_000240185_10 214478
gnl|BU-ISCIII|KPN05_000240185_11 206461
gnl|BU-ISCIII|KPN05_000240185_12 204006
gnl|BU-ISCIII|KPN05_000240185_13 192947
gnl|BU-ISCIII|KPN05_000240185_14 171545
gnl|BU-ISCIII|KPN05_000240185_15 131924
gnl|BU-ISCIII|KPN05_000240185_16 130978
gnl|BU-ISCIII|KPN05_000240185_17 129561
gnl|BU-ISCIII|KPN05_000240185_18 118948


In [144]:
#Now chop into files
#Credits: https://biopython.org/wiki/Split_large_file
#for i, batch in enumerate(batch_iterator(fasta_to_chop, 100000)):
#    header = record_to_chop.id
#    new_header = "%_%i.fastq" % (header, (i + 1))
#    with open(new_header, "w") as handle:
#        count = SeqIO.write(batch, handle, "fasta")
#    print("Wrote %i records to %s" % (count, new_header))

fasta_to_chop = SeqIO.parse(open("TEST/KPN_FULL.fna"),"fasta")

#Reset file if exist
fasta_to_dump = open("TEST/KPN_FULL_100000.fna", 'w').close()

fasta_to_dump = open("TEST/KPN_FULL_100000.fna", "a")

for record_to_chop in fasta_to_chop:
    
    fasta_header = str(record_to_chop.id)
    
    
    if len(record_to_chop) > 100000:
        split_times = round(len(record_to_chop) / 100000)
        split_remaining = len(record_to_chop) % 100000
        last_chop_legth = (100000 + split_remaining)
        
        
        total_splits = split_times + 1
        
        initial_position = 0
        final_position = 100000
        
        for time_to_split in range(1, total_splits):
            
            
            if time_to_split > 1 and time_to_split < split_times:
                initial_position = initial_position + 100000
                final_position = final_position + 100000
            elif time_to_split == split_times and split_times == 1:
                initial_position = 0
                final_position = final_position + split_remaining + 1
            elif time_to_split == split_times:
                initial_position = initial_position + 100000
                final_position = final_position + last_chop_legth + 1
                
            #print(initial_position, final_position)
            
            new_fasta_header = ">" + fasta_header + "_" + str(time_to_split)
            new_fasta_sequence = record_to_chop.seq[initial_position:final_position]
            
                          
            fasta_to_dump.write("%s\n%s\n" % (new_fasta_header, new_fasta_sequence))
            #fasta_to_dump.write("%s\n" % new_fasta_header)
                  
            #print(new_fasta_header, len(new_fasta_sequence), split_times, last_chop_legth, len(record_to_chop))
            #print(record_to_chop.seq[initial_position:final_position])
    else:
        last_chop_legth = 0
        
        fasta_to_dump.write(">%s\n%s\n" % (fasta_header, record_to_chop.seq))
        #print(fasta_header, len(record_to_chop), split_times, last_chop_legth, len(record_to_chop))
                  
                 
fasta_to_dump.close()
        
        #print("The whole length is %s" % len(record_to_chop))
        #print("The record %s will be chopped in %s sequences with the last one having %sbp" % (fasta_header, split_times, last_chop_legth))        
        
        #print(repr(fasta_to_chop.seq))

In [ ]:
#check that the new file has all the sequences
new_fasta = SeqIO.parse(open("TEST/KPN_FULL_100000.fna"),"fasta")
for record in new_fasta:
    print(record.id, len(record))

In [20]:
blast_test = SeqIO.parse(open("TEST/TEST_BLAST.fasta"),"fasta")
for record in blast_test:
    print(record.id, len(record))

    result_handle = NCBIWWW.qblast("blastn","nr", record, hitlist_size=5, alignments=10 word_size=20)

    blast_qresult = SearchIO.read(result_handle, "blast-xml")
    
    
    for hit in blast_qresult:
        print(hit)

print("\n\n")


'''
blast_record = NCBIXML.read(result_handle)
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        
        print('****Alignment****')
        print('sequence:', alignment.title)
        print('length:', alignment.length)
        print('e value:', hsp.expect)
        print(hsp.query[0:75] + '...')
        print(hsp.match[0:75] + '...')
        print(hsp.sbjct[0:75] + '...')
        #f.write('****Alignment****\n')
        #f.write("sequence: "+ str(alignment.title) + "\n")
        #f.write("length: "+ str(alignment.length) + "\n")
        #f.write("e-value: "+ str(hsp.expect) + "\n")
        #f.write("Query: "+ str(hsp.query[0:75]) + "\n")
        #f.write("Match: "+ str(hsp.match[0:75]) + "\n")
        #f.write("Subject: "+ str(hsp.sbjct[0:75]) + "\n")
'''

gnl|BU-ISCIII|K9682_BLAST_74 1221
Query: No
       definition line
  Hit: gi|1584040503|gb|CP036307.1| (229483)
       Klebsiella pneumoniae strain WCHKP020098 plasmid p1_020098, complete ...
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0   3.4e-17      98.67      54         [54:108]        [120449:120503]
          1   3.4e-17      98.67      54         [54:108]        [224403:224457]
Query: No
       definition line
  Hit: gi|1566460827|gb|CP034406.1| (300327)
       Klebsiella pneumoniae strain NH34 plasmid pNH34.1, complete sequence
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
      

Query: No
       definition line
  Hit: gi|1519001523|gb|CP033870.1| (6667)
       Acinetobacter baumannii strain MRSN15313 plasmid p597A-6.7, complete ...
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0   3.4e-17      98.67      54         [54:108]            [5114:5168]
Query: No
       definition line
  Hit: gi|1420776156|gb|CP030109.1| (84678)
       Acinetobacter baumannii strain DA33382 plasmid pDA33382-85, complete ...
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0   3.4e-17      98.67      54         [54:108]          [79736:79790]
Query: No
       definition line
  Hit:

'\nblast_record = NCBIXML.read(result_handle)\nfor alignment in blast_record.alignments:\n    for hsp in alignment.hsps:\n        \n        print(\'****Alignment****\')\n        print(\'sequence:\', alignment.title)\n        print(\'length:\', alignment.length)\n        print(\'e value:\', hsp.expect)\n        print(hsp.query[0:75] + \'...\')\n        print(hsp.match[0:75] + \'...\')\n        print(hsp.sbjct[0:75] + \'...\')\n        #f.write(\'****Alignment****\n\')\n        #f.write("sequence: "+ str(alignment.title) + "\n")\n        #f.write("length: "+ str(alignment.length) + "\n")\n        #f.write("e-value: "+ str(hsp.expect) + "\n")\n        #f.write("Query: "+ str(hsp.query[0:75]) + "\n")\n        #f.write("Match: "+ str(hsp.match[0:75]) + "\n")\n        #f.write("Subject: "+ str(hsp.sbjct[0:75]) + "\n")\n'